# 14.32 final paper

## Setup

In [1]:
import pandas as pd
import numpy as np
import microdf as mdf

In [2]:
mdf.set_plot_style()

## Load data

In [3]:
cases_raw = pd.read_csv('https://github.com/nytimes/covid-19-data/raw/master/us-counties.csv')

In [4]:
closures_raw = pd.read_csv('https://raw.githubusercontent.com/Keystone-Strategy/covid19-intervention-data/master/complete_npis_inherited_policies.csv')

In [5]:
pop_raw = pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv',
                      usecols=['STATE', 'COUNTY', 'POPESTIMATE2019'])

In [6]:
cps_raw = pd.read_csv('cps.csv.gz')

## Preprocess

### Population

In [7]:
pop = pop_raw.copy(deep=True)
pop['county_padded'] = pop.COUNTY.apply(lambda x: str(x).zfill(3))
pop['fips'] = (pop.STATE.astype(str) + pop.county_padded).astype(int)

In [8]:
county_pop = pop[pop.COUNTY > 0][['fips', 'POPESTIMATE2019']]
state_pop = pop[pop.COUNTY == 0][['STATE', 'POPESTIMATE2019']]
state_pop.rename({'STATE': 'fips'}, axis=1, inplace=True)

### School closures

In [9]:
closures = closures_raw[
    (closures_raw.npi == 'school_closure') & ~closures_raw.county.isna()][
    ['fip_code', 'start_date']]

In [10]:
closures.start_date = pd.to_datetime(closures.start_date)
closures.columns = ['fips', 'school_closure_date']

In [11]:
first_closure_date = closures.school_closure_date.min()
last_closure_date = closures.school_closure_date.max()
print("Schools closed between " + str(first_closure_date) + " and " + 
      str(last_closure_date))

Schools closed between 2020-03-12 00:00:00 and 2020-04-02 00:00:00


In [12]:
county = county_pop.merge(closures, how='left', on='fips')

In [13]:
county.to_csv('data/county.csv', index=False)

### Cases

In [14]:
cases = cases_raw[cases_raw.date >= '2020-03-01'][
    ['date', 'fips', 'cases', 'deaths']].copy(deep=True)

In [15]:
cases.date = pd.to_datetime(cases.date)

In [16]:
county_date = cases.merge(county, on='fips')

In [17]:
county_date['cases_pc'] = (1e6 * county_date.cases /
                           county_date.POPESTIMATE2019)
county_date['deaths_pc'] = (1e6 * county_date.deaths /
                            county_date.POPESTIMATE2019)

Add 1 since deaths are zero sometimes, and multiply by 1 million for variation in the feature.

In [18]:
county_date['log_cases_pc'] = np.log(county_date.cases_pc + 1)
county_date['log_deaths_pc'] = np.log(county_date.deaths_pc + 1)

In [19]:
county_date.to_csv('data/county_date.csv', index=False)

### Merge

In [20]:
cps = cps_raw.merge(county.rename({'fips': 'COUNTY'}, axis=1),
                    how='left', on='COUNTY')

In [21]:
cps['female'] = cps.SEX == 2

Add date corresponding to the end of each month's survey week.

In [22]:
cps['day'] = 12
cps['cps_date'] = pd.to_datetime(cps[['YEAR', 'MONTH', 'day']])
# Day of the week to find the following Saturday.
# Series.dt.weekday is 0 for Monday, 6 for Sunday.
cps['cps_weekday'] = cps.cps_date.dt.weekday
cps['days_to_sat'] = np.where(cps.cps_weekday == 6, 12, 5) - cps.cps_weekday
cps['cps_end_date'] = cps.cps_date + cps.days_to_sat.astype('timedelta64[D]')
cps['cps_start_date'] = cps.cps_end_date - pd.DateOffset(days=6)
# Drop unnecessary intermediate columns.
cps.drop(['day', 'cps_date', 'cps_weekday', 'days_to_sat'], axis=1, inplace=True)

Find two most recent CPS survey weeks.

In [23]:
cps.drop_duplicates('cps_end_date').sort_values('cps_end_date').tail(
    2)[['cps_start_date', 'cps_end_date']]

,cps_start_date,cps_end_date
1001950,2020-03-08,2020-03-14
1040952,2020-04-12,2020-04-18


https://cps.ipums.org/cps-action/variables/EMPSTAT#codes_section

In [24]:
cps['unemp'] = cps.EMPSTAT.isin([20, 21, 22])
cps['emp'] = cps.EMPSTAT.isin([10, 12])
cps['lf'] = cps.unemp | cps.emp

Some oversampled people have zero weight.

In [25]:
cps = cps[cps.WTFINL > 0]

Define kids age 6 to 18.

In [26]:
cps['has_k6'] = cps.NCHILD > cps.NCHLT5

Define days since schools closed (based on the end of the period) and post
flag from that.

*NB: This will not be the `post` flag used for the simple DDs, where the only
flag is April and all CPS persons are included, not only those with valid counties.*

In [27]:
cps['days_sc'] = np.maximum(
    (cps.cps_end_date - cps.school_closure_date).dt.days, 0)
cps['days_sc_has_k6'] = cps.days_sc * cps.has_k6

In [28]:
cps['post'] = cps.days_sc > 0
cps['post_has_k6'] = cps.post & cps.has_k6

In [29]:
cps['days_since_2000'] = (
    cps.cps_end_date - pd.to_datetime('2000-01-01')).dt.days

Needs to be mapped per
https://cps.ipums.org/cps-action/variables/FAMINC#codes_section.

In [30]:
#cps['other_faminc'] = cps.FAMINC - cps.EARNWEEK * 52 # * UHRSWORKT / AHRSWORKT
#cps['log_other_faminc'] = 

In [31]:
cps.FAMINC.value_counts()

842    184392
843    174936
841    155480
830    117775
820     87045
740     76839
720     49796
730     49559
710     38010
600     37382
500     25751
100     20133
430     16837
470     15062
300     12252
210      9422
Name: FAMINC, dtype: int64

For easier analysis.

In [33]:
cps.rename({'WTFINL': 'w', 'AHRSWORKT': 'hours', 'AGE': 'age'}, 
           axis=1, inplace=True)

Set NIUs to null.

In [34]:
cps.EARNWEEK = np.where(cps.EARNWEEK == 9999.99, np.nan, cps.EARNWEEK)
cps.hours = np.where(cps.hours == 999, np.nan, cps.hours)
# If excluding NILF.
cps['emp_of_lf'] = np.where(cps.lf, cps.emp, np.nan)

Log weekly earnings.

In [35]:
cps['lwe'] = np.log(cps.EARNWEEK)

Flags for simple unclustered regression.

In [36]:
cps['apr2020'] = (cps.YEAR == 2020) & (cps.MONTH == 4)
cps['apr2020_has_k6'] = cps.apr2020 & cps.has_k6

Triple difference fields and associated sub-interactions.

In [40]:
cps['apr2020_has_k6_female'] = cps.apr2020_has_k6 & cps.female
cps['apr2020_female'] = cps.apr2020 & cps.female
cps['has_k6_female'] = cps.has_k6 & cps.female

cps['days_sc_has_k6_female'] = cps.days_sc_has_k6 * cps.female
cps['days_sc_female'] = cps.days_sc * cps.female

In [38]:
# Do with other_faminc

# cps['apr2020_has_k6_female'] = cps.apr2020_has_k6 * cps.female
# cps['apr2020_female'] = cps.apr2020 * cps.female
# cps['has_k6_female'] = cps.has_k6 * cps.female

# cps['days_sc_has_k6_female'] = cps.days_sc_has_k6 * cps.female
# cps['days_sc_female'] = cps.days_sc * cps.female

Age squared.

In [41]:
cps['age2'] = cps.age * cps.age

Set married flag per https://cps.ipums.org/cps-action/variables/MARST#codes_section.

In [42]:
cps['married'] = cps.MARST.isin([1, 2])

Export.

In [45]:
OUTCOLS = ['YEAR', 'MONTH', 'COUNTY', 'w', 'age', 'age2', 'female', 'married',
           'hours', 'POPESTIMATE2019', 'school_closure_date', 
           'cps_start_date', 'cps_end_date', 'unemp', 'emp', 'lf', 'emp_of_lf',
           'has_k6', 'days_sc', 'post', 'post_has_k6', 'lwe',
           'days_since_2000', 'apr2020', 'apr2020_has_k6', 'days_sc_has_k6',
           'apr2020_has_k6_female', 'apr2020_female', 'has_k6_female',
           'days_sc_has_k6_female', 'days_sc_female']

In [46]:
cps[OUTCOLS].to_csv('data/cps.csv', index=False)

## First stage

In [ ]:
cases['school_closure_date_num'] = (
    cases.school_closure_date - first_closure_date).dt.days
cases = sm.add_constant(cases)

In [ ]:
def get_t(cases, var):
    regs = sm.WLS(cases.school_closure_date_num,
                  cases[[var, 'const']],
                  weights=cases.POPESTIMATE2019).fit().summary()
    return float(regs.tables[1][1].data[3])

In [ ]:
def get_ts(cases):
    return pd.Series([get_t(cases, 'cases'), get_t(cases, 'deaths'),
                      get_t(cases, 'cases_pc'), get_t(cases, 'deaths_pc'),
                      get_t(cases, 'log_cases_pc'),
                      get_t(cases, 'log_deaths_pc')],
                     index=['cases', 'deaths', 'cases_pc', 'deaths_pc',
                            'log_cases_pc', 'log_deaths_pc'])

In [ ]:
fs_df = cases[cases.date.between('2020-03-01', '2020-03-31')].groupby(
    'date').apply(get_ts)

In [ ]:
fs_df

In [ ]:
ax = fs_df.plot()
plt.title('t statistic of county regressions of Covid-19 on school closure date')
ax.axvline('2020-03-12', c='lightgray')
plt.xlabel('Date of cumulative Covid-19 cases and deaths')
plt.ylabel('t statistic of county regressions on school closure date')
plt.legend(['Covid-19 cases', 'Covid-19 deaths',
            'Cases per capita', 'Deaths per capita',
            'Log cases per capita', 'Log deaths per capita'])
plt.show()

In [ ]:
fs_df.loc[:first_closure_date].iloc[:-1]

Use Covid-19 deaths per capita on the date before the first school closure.

In [ ]:
COVID_DT = '2020-03-11'

In [ ]:
cases_covid_dt = cases[cases.date == COVID_DT]

In [ ]:
sm.WLS(cases_covid_dt.school_closure_date_num,
       cases_covid_dt[['deaths_pc', 'const']],
       cases_covid_dt.POPESTIMATE2019).fit().summary()

That is, each Covid-19 death per million population as of 2020-03-11 is
associated with schools being closed 0.35 days earlier.

Look at full set of Covid features to consider multiple instruments.

In [ ]:
sm.WLS(cases_covid_dt.school_closure_date_num,
       cases_covid_dt[['cases', 'cases_pc', 'log_cases_pc',
                       'deaths', 'deaths_pc', 'log_deaths_pc', 'const']],
       cases_covid_dt.POPESTIMATE2019).fit().summary()

In [ ]:
cases_covid_dt.plot.scatter(x='deaths_pc', y='school_closure_date')
plt.title('First stage: School closure date ~ deaths per capita by county')
plt.xlabel('Covid-19 deaths per million population as of 2020-03-11')
plt.ylabel('School closure date')
plt.show()

Apply fixed effects by de-meaning, to be able to stay with `statsmodels` rather than
`linearmodels` which doesn't work with `stargazer`.

See https://stackoverflow.com/a/24196288/1840471:

```
   ybar = y.mean()
    y = y -  y.groupby(data[absorb]).transform('mean') + ybar

    Xbar = X.mean()
    X = X - X.groupby(data[absorb]).transform('mean') + Xbar

    reg = sm.OLS(y,X)
    # Account for df loss from FE transform
    reg.df_resid -= (data[absorb].nunique() - 1)
```

## Instrument

Instrument for school closure using employment rate of mothers as of January 2020.

In [ ]:
cps['mother'] = cps.female & cps.has_k6
cps['emp_mother'] = cps.mother & cps.emp

In [ ]:
cps_jan2020_mothers = cps[(cps.YEAR == 2020) & (cps.MONTH == 1) & cps.mother]

In [ ]:
county_mother_emp_jan2020 = cps_jan2020_mothers.groupby('COUNTY').apply(
    lambda x: mdf.weighted_mean(x, 'emp_mother', 'WTFINL')
)

In [ ]:
county_mother_emp_jan2020

In [ ]:
county_mother_emp_jan2020 = pd.DataFrame(county_mother_emp_jan2020)
county_mother_emp_jan2020.columns = ['mother_emp_jan2020']

In [ ]:
closures_mother_emp = closures.merge(county_mother_emp_jan2020, on='COUNTY')

In [ ]:
closures_mother_emp.plot.scatter(x='mother_emp_jan2020', y='school_closure_date')
plt.title('First stage: School closure date ~ mother employment rate by county')
plt.xlabel('Employment rate of mothers as of January 2020')
plt.ylabel('School closure date')
plt.show()

In [ ]:
closures_mother_emp['school_closure_date_num'] = (
    closures_mother_emp.school_closure_date - first_closure_date).dt.days
closures_mother_emp = sm.add_constant(closures_mother_emp)

In [ ]:
closures_mother_emp.describe()

In [ ]:
sm.OLS(closures_mother_emp.school_closure_date_num,
       closures_mother_emp[['mother_emp_jan2020', 'const']]).fit().summary()

In [ ]:
closures_mother_emp.corr()

## Regressions

In [ ]:
cps[cps.post].cps_date.value_counts()

In [ ]:
cps.COUNTY.isna().sum()

In [ ]:
cps.COUNTY.value_counts()

In [ ]:
cps.groupby(['has_k6', 'post']).size()

In [ ]:
cps.columns

In [ ]:
mdf.weighted_mean(cps[cps.lf & (cps.YEAR == 2020) & (cps.MONTH == 3)], 'emp', 'WTFINL')

In [ ]:
mdf.weighted_mean(cps[cps.lf & (cps.YEAR == 2020) & (cps.MONTH == 4)], 'emp', 'WTFINL')

In [ ]:
XCOLS = ['has_k6', 'post', 'post_has_k6', 'female', 'const', 'cps_date_float']
sm.WLS(cps.EARNWEEK, cps[XCOLS].astype('float'), weights=cps.WTFINL).fit().summary()

In [ ]:
XCOLS = ['has_k6', 'post', 'post_has_k6', 'female', 'const', 'cps_date_float']
sm.WLS(cps.EARNWEEK, cps[XCOLS].astype('float'), weights=cps.WTFINL).fit(
    cov_type='cluster', cov_kwds={'groups': cps.COUNTY}).summary()